## Ajuste lineal

In [9]:
import numpy as np

def ajuste_lineal_minimos_cuadrados(x, y):
    """
    Realiza un ajuste lineal por mínimos cuadrados y retorna la expresión de la recta.

    Parámetros:
    x : array_like
        Lista o arreglo de valores de x.
    y : array_like
        Lista o arreglo de valores de y correspondientes.

    Retorna:
    a0, a1 : float
        Coeficientes de la recta ajustada: y = a0 + a1 * x
    expresion : str
        Representación en cadena de la recta ajustada
    """
    x = np.array(x)
    y = np.array(y)

    N = len(x)
    M = np.vstack((np.ones(N), x)).T

    MTM = M.T @ M
    MTy = M.T @ y
    a = np.linalg.solve(MTM, MTy)

    a0, a1 = a
    expresion = f"y = {a0:.4f} + {a1:.4f}x"
    return a0, a1, expresion

# Ejemplo de uso:
x = [1, 2, 3, 4, 5]
y = [2.2, 2.8, 3.6, 4.5, 5.1]
a0, a1, expresion = ajuste_lineal_minimos_cuadrados(x, y)
print(expresion)


y = 1.3900 + 0.7500x


In [10]:
import numpy as np

def base_spline_lineal(x, nodo):
    """
    Construye la matriz de diseño M usando base lineal con un término de tipo (x - nodo)_+.
    
    Parámetros:
    x : array_like
        Valores de entrada.
    nodo : float
        Punto a partir del cual se activa la rampa (x - nodo)_+.
    
    Retorna:
    M : ndarray
        Matriz de diseño con columnas [1, x, (x - nodo)_+].
    """
    x = np.array(x)
    columna_constante = np.ones_like(x)
    columna_x = x
    columna_rampa = np.maximum(0, x - nodo)
    return np.vstack((columna_constante, columna_x, columna_rampa)).T

def ajuste_spline_lineal_personalizado(x, y, nodo):
    """
    Realiza el ajuste spline lineal con un término (x - nodo)_+ como base.
    
    Parámetros:
    x : array_like
        Valores de entrada.
    y : array_like
        Valores de salida.
    nodo : float
        Nodo de activación para la base (x - nodo)_+.
    
    Retorna:
    a0, a1, a2 : float
        Coeficientes ajustados del modelo: y = a0 + a1 * x + a2 * (x - nodo)_+
    """
    M = base_spline_lineal(x, nodo)
    MTM = M.T @ M
    MTy = M.T @ y
    coeficientes = np.linalg.solve(MTM, MTy)
    return coeficientes

# Ejemplo de uso con los datos del problema
x = np.array([500, 700, 900, 1100, 1300, 1500, 1700, 1900])
y = np.array([365, 361.6, 370.64, 379.68, 384.46, 395.5, 395.95, 372.91])
nodo = 1100

# Ajustar modelo
a0, a1, a2 = ajuste_spline_lineal_personalizado(x, y, nodo)
print(f"Modelo ajustado:\n y = {a0:.4f} + {a1:.4f}x + {a2:.4f}(x - {nodo})_+")

# Predicción en Q = 1000
x_test = 1000
y_test = a0 + a1 * x_test + a2 * max(0, x_test - nodo)
print(f"Predicción para Q = 1000: y ≈ {y_test:.2f} W")

if y_test <= 373:
    print("✅ La bomba SÍ puede operar dentro del límite de 373 W.")
else:
    print("❌ La bomba NO puede operar dentro del límite de 373 W.")


Modelo ajustado:
 y = 339.9822 + 0.0392x + -0.0352(x - 1100)_+
Predicción para Q = 1000: y ≈ 379.18 W
❌ La bomba NO puede operar dentro del límite de 373 W.


In [11]:
import numpy as np

def ajuste_polinomial(x, y, grado):
    """
    Realiza un ajuste polinomial de grado especificado usando álgebra matricial.

    Parámetros:
    x : array_like
        Lista de valores de entrada.
    y : array_like
        Lista de valores de salida correspondientes.
    grado : int
        Grado del polinomio a ajustar.

    Retorna:
    coeficientes : ndarray
        Coeficientes del polinomio desde a_0 hasta a_n.
    expresion : str
        Expresión del polinomio en forma legible.
    """
    x = np.array(x)
    y = np.array(y)

    # Matriz de Vandermonde
    M = np.vander(x, N=grado+1, increasing=True)

    # Resolver (MᵀM)a = Mᵀy
    MTM = M.T @ M
    MTy = M.T @ y
    coeficientes = np.linalg.solve(MTM, MTy)

    # Construir la expresión legible
    expresion = "y = " + " + ".join(
        [f"{coef:.4f}x^{i}" if i > 0 else f"{coef:.4f}" for i, coef in enumerate(coeficientes)]
    )
    return coeficientes, expresion

# Ejemplo de uso con datos del problema
x = np.array([500, 700, 900, 1100, 1300, 1500, 1700, 1900])
y = np.array([365, 361.6, 370.64, 379.68, 384.46, 395.5, 395.95, 372.91])

grado = 4  # Puedes cambiar el grado aquí
coef, expr = ajuste_polinomial(x, y, grado)

print("Modelo ajustado:")
print(expr)

# Evaluación en x = 1000
x_test = 1000
y_test = sum(c * x_test**i for i, c in enumerate(coef))
print(f"Predicción para x = {x_test}: y ≈ {y_test:.2f} W")

if y_test <= 373:
    print("✅ La bomba SÍ puede operar dentro del límite de 373 W.")
else:
    print("❌ La bomba NO puede operar dentro del límite de 373 W.")


Modelo ajustado:
y = 358.0671 + 0.0155x^1 + 0.0000x^2 + 0.0000x^3 + 0.0000x^4
Predicción para x = 1000: y ≈ 374.27 W
❌ La bomba NO puede operar dentro del límite de 373 W.


## Ajuste no lineal

In [12]:
import numpy as np

def ajuste_exponencial(x, y):
    """
    Ajuste exponencial del tipo y = a * exp(b * x) usando matrices.

    Parámetros:
    x : array_like
        Valores de entrada.
    y : array_like
        Valores de salida (deben ser positivos).

    Retorna:
    a, b : float
        Coeficientes del modelo exponencial.
    expresion : str
        Ecuación ajustada como string.
    """
    x = np.array(x)
    y = np.array(y)

    if np.any(y <= 0):
        raise ValueError("Todos los valores de y deben ser positivos para ajuste exponencial.")

    # Transformación logarítmica
    log_y = np.log(y)

    # Matriz de diseño para log(y) = ln(a) + b * x
    M = np.vstack((np.ones_like(x), x)).T

    # Mínimos cuadrados: (M^T M) a = M^T y
    MTM = M.T @ M
    MTy = M.T @ log_y
    coef = np.linalg.solve(MTM, MTy)

    ln_a, b = coef
    a = np.exp(ln_a)

    expresion = f"y = {a:.4f} * e^({b:.4f}x)"
    return a, b, expresion

# Ejemplo de uso
x = np.array([500, 700, 900, 1100, 1300, 1500, 1700, 1900])
y = np.array([365, 361.6, 370.64, 379.68, 384.46, 395.5, 395.95, 372.91])

a, b, expr = ajuste_exponencial(x, y)

print("Modelo ajustado:")
print(expr)

# Evaluación en un punto
x_test = 1000
y_test = a * np.exp(b * x_test)
print(f"Predicción para x = {x_test}: y ≈ {y_test:.2f}")

if y_test <= 373:
    print("✅ El modelo predice que se está dentro del límite.")
else:
    print("❌ El modelo predice que se excede el límite.")


Modelo ajustado:
y = 356.7432 * e^(0.0000x)
Predicción para x = 1000: y ≈ 374.39
❌ El modelo predice que se excede el límite.


In [13]:
import numpy as np

def ajuste_potencial(x, y):
    """
    Ajuste de modelo potencial del tipo y = a * x^b usando producto de matrices.

    Parámetros:
    x : array_like
        Valores de entrada (positivos).
    y : array_like
        Valores de salida (positivos).

    Retorna:
    a, b : float
        Coeficientes del modelo.
    expresion : str
        Expresión ajustada como string.
    """
    x = np.array(x)
    y = np.array(y)

    if np.any(x <= 0) or np.any(y <= 0):
        raise ValueError("x e y deben ser positivos para el ajuste potencial.")

    log_x = np.log(x)
    log_y = np.log(y)

    # Matriz de diseño: columnas [1, ln(x)]
    M = np.vstack((np.ones_like(log_x), log_x)).T

    # Resolver el sistema normal (M^T M) c = M^T log_y
    MTM = M.T @ M
    MTy = M.T @ log_y
    coef = np.linalg.solve(MTM, MTy)

    ln_a, b = coef
    a = np.exp(ln_a)

    expresion = f"y = {a:.4f} * x^{b:.4f}"
    return a, b, expresion

# Ejemplo de uso
x = np.array([500, 700, 900, 1100, 1300, 1500, 1700, 1900])
y = np.array([365, 361.6, 370.64, 379.68, 384.46, 395.5, 395.95, 372.91])

a, b, expr = ajuste_potencial(x, y)

print("Modelo ajustado:")
print(expr)

# Evaluación en un punto
x_test = 1000
y_test = a * x_test**b
print(f"Predicción para x = {x_test}: y ≈ {y_test:.2f}")

if y_test <= 373:
    print("✅ El modelo predice que se está dentro del límite.")
else:
    print("❌ El modelo predice que se excede el límite.")


Modelo ajustado:
y = 257.2441 * x^0.0549
Predicción para x = 1000: y ≈ 376.00
❌ El modelo predice que se excede el límite.


In [14]:
import numpy as np

def ajuste_racional(x, y):
    """
    Ajuste de modelo racional del tipo y = 1 / (a * x + b) usando mínimos cuadrados.

    Parámetros:
    x : array_like
        Valores de entrada.
    y : array_like
        Valores de salida (no deben ser cero).

    Retorna:
    a, b : float
        Coeficientes del modelo racional.
    expresion : str
        Ecuación ajustada como string.
    """
    x = np.array(x)
    y = np.array(y)

    if np.any(y == 0):
        raise ValueError("y no puede contener ceros para ajuste racional.")

    z = 1 / y  # Transformamos el modelo: 1/y = a*x + b

    M = np.vstack((x, np.ones_like(x))).T
    MTM = M.T @ M
    MTz = M.T @ z
    coef = np.linalg.solve(MTM, MTz)

    a, b = coef
    expresion = f"y = 1 / ({a:.4f}x + {b:.4f})"
    return a, b, expresion

# Ejemplo de uso
x = np.array([500, 700, 900, 1100, 1300, 1500, 1700, 1900])
y = np.array([365, 361.6, 370.64, 379.68, 384.46, 395.5, 395.95, 372.91])

a, b, expr = ajuste_racional(x, y)

print("Modelo ajustado:")
print(expr)

# Evaluación en un punto
x_test = 1000
y_test = 1 / (a * x_test + b)
print(f"Predicción para x = {x_test}: y ≈ {y_test:.2f}")

if y_test <= 373:
    print("✅ El modelo predice que se está dentro del límite.")
else:
    print("❌ El modelo predice que se excede el límite.")


Modelo ajustado:
y = 1 / (-0.0000x + 0.0028)
Predicción para x = 1000: y ≈ 374.21
❌ El modelo predice que se excede el límite.


## Comparar errores

In [15]:
def error_ajuste(y_real, y_estimado):
    """
    Calcula el error cuadrático medio E del modelo.

    Parámetros:
    y_real : array_like
        Valores reales de y.
    y_estimado : array_like
        Valores ajustados del modelo.

    Retorna:
    float
        Error cuadrático medio E.
    """
    y_real = np.array(y_real)
    y_estimado = np.array(y_estimado)
    N = len(y_real)
    E = np.sqrt(np.sum((y_real - y_estimado)**2) / N)
    return E


In [16]:
# ✅ Datos del problema
x = np.array([500, 700, 900, 1100, 1300, 1500, 1700, 1900])
y = np.array([365, 361.6, 370.64, 379.68, 384.46, 395.5, 395.95, 372.91])

# ⚙️ POLINOMIAL
grado = 4
coef_poly, expr_poly = ajuste_polinomial(x, y, grado)
y_poly = sum(c * x**i for i, c in enumerate(coef_poly))
err_poly = error_ajuste(y, y_poly)

# ⚙️ EXPONENCIAL
a_exp, b_exp, expr_exp = ajuste_exponencial(x, y)
y_exp = a_exp * np.exp(b_exp * x)
err_exp = error_ajuste(y, y_exp)

# ⚙️ POTENCIAL
a_pot, b_pot, expr_pot = ajuste_potencial(x, y)
y_pot = a_pot * x**b_pot
err_pot = error_ajuste(y, y_pot)

# 📊 RESULTADOS
print("📊 Comparación de errores:")
print(f"- Polinomial (grado {grado}):")
print(f"  {expr_poly}")
print(f"  Error: {err_poly:.4f}")

print(f"\n- Exponencial:")
print(f"  {expr_exp}")
print(f"  Error: {err_exp:.4f}")

print(f"\n- Potencial:")
print(f"  {expr_pot}")
print(f"  Error: {err_pot:.4f}")


📊 Comparación de errores:
- Polinomial (grado 4):
  y = 358.0671 + 0.0155x^1 + 0.0000x^2 + 0.0000x^3 + 0.0000x^4
  Error: 9.1427

- Exponencial:
  y = 356.7432 * e^(0.0000x)
  Error: 8.9349

- Potencial:
  y = 257.2441 * x^0.0549
  Error: 8.3405


## Grado


In [3]:
import numpy as np
import pandas as pd

def diferencias_divididas(x, y):
    """
    Calcula la tabla de diferencias divididas.
    
    Parámetros:
    x, y : array_like
        Vectores de entrada.

    Retorna:
    tabla : pandas.DataFrame
        Tabla con las diferencias divididas.
    """
    x = np.array(x, dtype=float)
    y = np.array(y, dtype=float)
    N = len(x)

    tabla = np.zeros((N, N))
    tabla[:, 0] = y

    # Llenar tabla de diferencias divididas
    for j in range(1, N):
        for i in range(N - j):
            tabla[i, j] = (tabla[i + 1, j - 1] - tabla[i, j - 1]) / (x[i + j] - x[i])

    # Crear tabla como DataFrame para mejor presentación
    columnas = ['f[x]'] + [f'D.D. {j}' for j in range(1, N)]
    df = pd.DataFrame(tabla, columns=columnas)
    df.insert(0, 'x', x)
    return df

def sugerir_grado(tabla, tolerancia=1e-4):
    """
    Sugerencia de grado a partir de diferencias divididas que se vuelven casi constantes o pequeñas.

    Retorna:
    grado_sugerido : int
        El menor grado donde las diferencias de ese orden se vuelven casi constantes.
    """
    for j in range(2, tabla.shape[1]):
        col = tabla.iloc[:, j].dropna().values
        if np.all(np.abs(np.diff(col)) < tolerancia):
            return j - 1  # j-1 porque D.D. 2 → grado 2
    return tabla.shape[1] - 2  # Si no hay diferencias estables, asumir grado máximo posible

# Datos del ejemplo
x = np.array([500, 700, 900, 1100, 1300, 1500, 1700, 1900])
y = np.array([365, 361.6, 370.64, 379.68, 384.46, 395.5, 395.95, 372.91])

# Calcular tabla
tabla_dd = diferencias_divididas(x, y)
print("\n📋 Tabla de Diferencias Divididas:")
print(tabla_dd.round(10))

# Sugerir grado
grado = sugerir_grado(tabla_dd)
print(f"\n🔎 Sugerencia: usar un ajuste polinomial de grado {grado}.")



📋 Tabla de Diferencias Divididas:
        x    f[x]   D.D. 1    D.D. 2        D.D. 3        D.D. 4  D.D. 5  \
0   500.0  365.00 -0.01700  0.000156 -2.592000e-07  2.000000e-10     0.0   
1   700.0  361.60  0.04520  0.000000 -8.880000e-08  4.000000e-10    -0.0   
2   900.0  370.64  0.04520 -0.000053  2.192000e-07 -7.000000e-10     0.0   
3  1100.0  379.68  0.02390  0.000078 -3.510000e-07  1.000000e-10     0.0   
4  1300.0  384.46  0.05520 -0.000132 -2.687000e-07  0.000000e+00     0.0   
5  1500.0  395.50  0.00225 -0.000294  0.000000e+00  0.000000e+00     0.0   
6  1700.0  395.95 -0.11520  0.000000  0.000000e+00  0.000000e+00     0.0   
7  1900.0  372.91  0.00000  0.000000  0.000000e+00  0.000000e+00     0.0   

   D.D. 6  D.D. 7  
0    -0.0     0.0  
1     0.0     0.0  
2     0.0     0.0  
3     0.0     0.0  
4     0.0     0.0  
5     0.0     0.0  
6     0.0     0.0  
7     0.0     0.0  

🔎 Sugerencia: usar un ajuste polinomial de grado 3.
